Image Path:

/home/june/.cache/kagglehub/datasets/borhanitrash/animal-image-classification-dataset/versions/1

In [1]:
import os
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision.transforms as transforms
import torchvision
from torch.utils.data import random_split


In [2]:
dir_path = "/home/june/.cache/kagglehub/datasets/borhanitrash/animal-image-classification-dataset/versions/1/Animals/"
cats = dir_path + "/cats"
dogs = dir_path + "/dogs"
snakes = dir_path + "/snakes"

In [3]:

# Get all image file paths for each animal
cat_paths = [os.path.join(cats, fname) for fname in os.listdir(cats) if os.path.isfile(os.path.join(cats, fname))]
dog_paths = [os.path.join(dogs, fname) for fname in os.listdir(dogs) if os.path.isfile(os.path.join(dogs, fname))]
snake_paths = [os.path.join(snakes, fname) for fname in os.listdir(snakes) if os.path.isfile(os.path.join(snakes, fname))]


image_paths = cat_paths + dog_paths + snake_paths
labels = [0] * len(cat_paths) + [1] * len(dog_paths) + [2] * len(snake_paths)  # 0: cat, 1: dog, 2: snake
class_names = ['cat', 'dog', 'snake']

# print(labels)



In [4]:
data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

class AnimalDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, labels, transform=data_transforms):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
        # No need to convert labels to tensor here; do it in __getitem__ for compatibility with PyTorch loss functions

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
            label = torch.tensor(self.labels[idx], dtype=torch.long)
        label = self.labels[idx]
        return image, label
    
    # Helper method to split dataset into train, val, and test sets
    def __split__(self, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, seed=42):
        assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, "Ratios must sum to 1"
        dataset_size = len(self)
        indices = list(range(dataset_size))
        if seed is not None:
            np.random.seed(seed)
        np.random.shuffle(indices)
        train_end = int(train_ratio * dataset_size)
        val_end = train_end + int(val_ratio * dataset_size)
        train_indices = indices[:train_end]
        val_indices = indices[train_end:val_end]
        test_indices = indices[val_end:]
        train_set = torch.utils.data.Subset(self, train_indices)
        val_set = torch.utils.data.Subset(self, val_indices)
        test_set = torch.utils.data.Subset(self, test_indices)
        return train_set, val_set, test_set

dataset = AnimalDataset(image_paths, labels, transform=data_transforms)








In [5]:
train_dataset, val_dataset, test_dataset = dataset.__split__(train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, seed=42)

print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(val_dataset)}")
print(f"Test size: {len(test_dataset)}")

Train size: 2100
Validation size: 450
Test size: 450


In [6]:
print(f"shape of data : {dataset[0][0].shape}")

shape of data : torch.Size([3, 256, 256])


In [7]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 3)  # 3 classes: cat, dog, snake

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 32 * 32)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [8]:
net = NeuralNet()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001,momentum=0.9)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=2)
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        # inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = net(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
            running_loss = 0.0

[1, 100] loss: 1.097
[1, 200] loss: 1.085


KeyboardInterrupt: 